In [90]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pickle

In [91]:
##Load the scalar, pkl file, model
model = load_model('ann_model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


with open('le_gender.pkl', 'rb') as f:
    le_gender = pickle.load(f)

with open('ohe_geo.pkl', 'rb') as f:
    ohe_geo = pickle.load(f)

In [110]:
# Example input data
input_data = {
    'CreditScore': 750,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 38,
    'Tenure': 7,
    'Balance': 75000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 95000
}

In [112]:
input_df = pd.DataFrame([input_data])
input_df['Gender'] = le_gender.transform(input_df['Gender'])
input_df





,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,750,Spain,0,38,7,75000,2,1,1,95000


In [113]:
ohe_geo_en = ohe_geo.transform([[input_data['Geography']]]).toarray()
ohe_geo_df = pd.DataFrame(ohe_geo_en, columns=ohe_geo.get_feature_names_out())
ohe_geo_df

c:\Users\Danns\OneDrive\Desktop\Udemy_GenAI\AnnClassification project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [115]:
input_df = pd.concat([input_df.drop('Geography', axis=1), ohe_geo_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,0,38,7,75000,2,1,1,95000,0.0,0.0,1.0


In [117]:
#Scaling
input_df_scaled = scaler.transform(input_df)

In [ ]:
#Predict churn
predictions = model.predict(input_df_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.04220176]], dtype=float32)

In [120]:
predictions_prob = predictions[0][0]
predictions_prob

np.float32(0.04220176)

In [123]:
if predictions_prob > 0.5:
    print(f'The customer is likely to churn with a probability of {predictions_prob:.2f}')
else:
    print(f'The customer is unlikely to churn with a probability of {predictions_prob:.2f}')    


The customer is unlikely to churn with a probability of 0.04
